## Airplane delays with data frames

In this notebook we show how to work with data frames in Apache Spark. We''ll be using the Airplane delays data set, which you can find [here](http://stat-computing.org/dataexpo/2009/the-data.html) in full, together with a description of the records.

We are interested in the delays of flights and we will be trying to predict them by using the machine learning model Random Forests. 

Before we start with the data we create a SparkSession. This is different from Sparks RDD API, where we had to create a SparkContext.

It is possible - and quite popular - to use both RDDs and data frames. However, data frames are much faster than RDDs and changing between this levels of abstractions can be quite confusing to someone new to Spark. Also, RDDs will be eventually fased out and will become deprecated in the future. 


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Airplane delays") \
    .getOrCreate()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook


### Getting the data
We assume the data is already in the data folder. If not you can download a by uncommenting the approprite lines in the cell below and execute it.

In [ ]:
#You can download the datafile by removing the comments in the next two lines
#import urllib
#f = urllib.urlretrieve("http://beehub.nl/surfsara-hadoop/public/2008.csv.gz", "data/2008.csv.gz")

### Inspecting the data

The file that we'll be using is zipped (using gzip). Its size is 174,6 MB and when unzipped it will be about 690 MB. This in itself is not a really large dataset but it will suffice for our example. 

The data file (once unzipped) starts with a line of headers, followed by data records on each line. They look like this (where the first line with the headers spills over three lines here.)  

<pre>
Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2008,1,3,4,2003,1955,2211,2225,WN,335,N712SW,128,150,116,-14,8,IAD,TPA,810,4,8,0,,0,NA,NA,NA,NA,NA
2008,1,3,4,754,735,1002,1000,WN,3231,N772SW,128,145,113,2,19,IAD,TPA,810,5,10,0,,0,NA,NA,NA,NA,NA
2008,1,3,4,628,620,804,750,WN,448,N428WN,96,90,76,14,8,IND,BWI,515,3,17,0,,0,NA,NA,NA,NA,NA
2008,1,3,4,926,930,1054,1100,WN,1746,N612SW,88,90,78,-6,-4,IND,BWI,515,3,7,0,,0,NA,NA,NA,NA,NA
2008,1,3,4,1829,1755,1959,1925,WN,3920,N464WN,90,90,77,34,34,IND,BWI,515,3,10,0,,0,2,0,0,0,32
2008,1,3,4,1940,1915,2121,2110,WN,378,N726SW,101,115,87,11,25,IND,JAX,688,4,10,0,,0,NA,NA,NA,NA,NA
2008,1,3,4,1937,1830,2037,1940,WN,509,N763SW,240,250,230,57,67,IND,LAS,1591,3,7,0,,0,10,0,0,0,47
2008,1,3,4,1039,1040,1132,1150,WN,535,N428WN,233,250,219,-18,-1,IND,LAS,1591,7,7,0,,0,NA,NA,NA,NA,NA
2008,1,3,4,617,615,652,650,WN,11,N689SW,95,95,70,2,2,IND,MCI,451,6,19,0,,0,NA,NA,NA,NA,NA </pre>

Note that this data file is structured. Also note that some of the fields have 'NA' as a value. This means 'Not Availaible' and is a bit of a nuisance. 'NA' is of type string and when Spark encounters a string as a possible value it will interpret the column as of type String. We will have to deal with this later. 

Now let's read the data into a Spark data frame. Will be using the [csv reader](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.csv) for this. Note that we do not have to unzip the file.

As an option we specify that the file contains headers (in the first line).
After loading the file in, we can print its schema, by using [printSchema](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.printSchema).

In [ ]:
#dfRaw = spark.read.option("header", "true").csv("data/2008.csv.gz", StructType(fields))
dfRawdata = spark.read.option("header", "true").csv("data/2008.csv.gz")
dfRawdata.printSchema()

All columns are of type string. It is possible to tell Spark what type the columns are before loading in the data. We did not do this, because we have to get rid of the 'NA' values.



### Inspecting the data  frame
Let's see what our data frame cointans by using [<i>show</i>](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.show). 

We will print out the first 10 records of the data frame. If we specify no arguments to show, it will print out 20 records.

In [ ]:
dfRawdata.show(10)

Print the columns of the data frame. You may want to check the [data frame](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame) API documentation.

In [ ]:
#TODO replace <fill in> with code of your own

# print the columns of this data frame

dfRawdata.<fill in>

### Counting the records

Let's see how many records we have. Again, you may want to consult the [API docs](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame). 


In [ ]:
#TODO replace <fill in> with code of your own

# print the number of records in this data frame

dfRawdata.<fill in>

Because we have quite a lot of records and we are in stand alone mode (not using more than one machine) let's take a sample of half the data set. We choose records at random and invoke the [sample](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.sample) method. 

In [ ]:
dfRaw = dfRawdata.sample(False, 0.5, 12345)

### Data munging

Let's get rid of the 'NA' fields in columns that we think are interesting. Note how we filter on multiple columns by chaining the calls to the [filter](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.filter) method.

After filtering we select only those fields that we think may contribute something to predicting the flight delays. Note that we include 'DepDelay', the departure delay. We will try to predict departure delays later in this notebook.

When selecting we also [cast](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast) the columns to the right type for our Random Forest model. We call printSchema on the new data frame to check the new types. 


In [ ]:
from pyspark.sql.types import IntegerType, LongType

dff = dfRaw.filter(dfRaw['DepDelay']!='NA').filter(dfRaw['Month']!='NA').filter(dfRaw['ArrDelay']!='NA') \
           .filter(dfRaw['UniqueCarrier']!='NA').filter(dfRaw['DayofMonth']!='NA').filter(dfRaw['DayofWeek']!='NA') \
            .filter(dfRaw['CRSDeptime']!= 'NA').filter(dfRaw['Origin']!='NA').filter(dfRaw['Dest']!='NA') \
           .filter(dfRaw['Distance']!='NA')

df = dff.select(dff['Month'].cast(LongType()), dff['DayofMonth'].cast(LongType()), 
                     dff['DayofWeek'].cast(LongType()), dff['DepDelay'].cast(LongType()),
                     dff['Origin'],dff['ArrDelay'].cast(LongType()),dff['CRSDepTime'].cast(LongType()),
                     dff['Distance'].cast(LongType()), dff['Dest'],dff['UniqueCarrier'])
df.printSchema()


As said before, a data frame is like a table. So let's see what it looks like by printing the columns and a single record. Below the [head](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.head) method is used, instead of show, which is for printing to he console.

We will use html to pretty print this table.

In [ ]:
printSample = df.head()
from IPython.display import display, HTML

th = ["<th>" + d + "</th>" for d in df.columns]
td = ["<td>" + str(d) + "</td>" for d in printSample]

display(HTML("<table><thead><tr>" + "".join(th) + "</tr></thead><tbody><tr>" + "".join(td) + "</tr></tbody></table>"))

### Basic descriptive statistics

We are interested in the departure delay. We can compute the number of records that lists this field, together with the mean value, the standard deviation, the minimum and the maximum values, by invoking [describe](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.describe) on the column. 

In [ ]:
df.select(df['DepDelay']).describe("DepDelay").show()

#  shorter and equivalent
#df[["DepDelay"]].describe("DepDelay").show()

### Data reduction - Average department delay by month

In the next cell we compute the average departure delay per month. We do this by selecting the columns 'DepDelay' and 'Month', then group by 'Month' and compute the mean of 'DepDelay'.

The result of the [groupby]([http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy) method is that for each month the values of 'DepDelay' are grouped together in a list. For each of these lists the mean is then computed.

Finally, we convert the data frame to a Python Pandas data frame. This means that we collect all data from the data frame to the driver. The Pandas data frame (called pdf here) is local, it will not be distributed over many machines when using a cluster. We will visualize this Pandas data frame later, hence the import of matplotlib which is a visualisation library. 

In [ ]:
# The import of matplotlib is for the graphics we use later

import matplotlib.pyplot as plt
%matplotlib inline

grouped = df[['DepDelay', 'Month']].groupby('Month').mean('DepDelay')

# The last line is equivalent to
# grouped = df.select(df['DepDelay'], df['Month]).groupby('Month').mean('DepDelay')

grouped_sorted = grouped.sort(grouped['Month'])

pdf = grouped_sorted.toPandas()


Let us see what the Pandas data frame looks like.

In [ ]:
pdf

### Visualisation of the Pandas data frame
It is easy to plot this data frame.

In [ ]:
pdf.plot(kind='bar', x='Month')

### Average department delay by carrier

Instead of the average delay per month we want to see the average delay per carrier. Please fill in the code at the appropriate place in the cell below.

Make sure you know the column that contains the value for the carrier. You can check above.

In [ ]:
# TO DO 
# Replace <FILL IN> with code of your own

grpd = <FILL IN>

carrier = grpd.sort('avg(DepDelay)', ascending=False)
cpd = carrier.toPandas()
cpd

Let's plot the resulting Pandas data frame.

In [ ]:
cpd.plot(kind='bar', x=cpd['UniqueCarrier'])

### Stats for the delay of AQ 

One airliner (AQ) is doing so well, it has a negative delay! This means that on average flights arrive early. Is this right? How many records are there for this airline (which is based in Hawaii) in our data frame? What is the mean, the standard deviation and the minumum and maximum delay for this carrier? 

Get this information for the aqs data frame in the cell below. Refer to previous cells for leads.

In [ ]:
#TODO
# Replace <FILL IN> with code of your own 
# one line should suffice

aqs = df.filter(df['UniqueCarrier'] == 'AQ')

#<FILL IN>
aqs[["DepDelay"]].<FILL IN>

Let's limit the prediction of departure delays to the airport in Chicago.

To do this we have to select all records in the df dataframe that have 'ORD' at the 'Origin' column.
In the resulting dataframe we can then drop the 'Origin' column, since all flights are from Chicago anyway and this information has become redundant.

You may want to look at the [drop method](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.drop) in the documentation.


In [ ]:
#TODO
# Replace <FILL IN> with code of your own 
# one line should suffice, two is possible

orddf.<FILL IN>
orddf.show()

## Machine Learning - predicting delays with Random Forests

In the rest of this notebook we will build a simple model that can be used to predict departure delays for a given airport.

We will define a delay as a departure delay of more than 15 minutes. This new feature is categorical and binary: a flight is either delayed or not. Then we will (after some data munging) train a model using Random Forests. This model can then be used to predict delays, based on new observations.

Here we will not explain what Random Forests are. For more info you may want to refer to [this](https://www.youtube.com/watch?v=3kYujfDgmNk) video. If you are not familiair with decision trees you may want to see [this one first](https://www.youtube.com/watch?v=-dCtJjlEEgM).

In order to train and test the model we will divide the data into a training set and a test set. 

This exercise is to show you how to use Machine Learning in Apache Spark. Obviously, the model that we'll build is not very special. The aim is here to show principles.

Apache Spark has two Machine Learning libraries, one for RDDs (MLLib) and one for Data Frames (ML). MLLib is gradually being fased out and new developments will centre around Spark.ml. 

For those familair with [Pythons Scikit-learn](http://scikit-learn.org/stable/): Spark.ml is very similar in design and also supports [pipelines](http://spark.apache.org/docs/latest/ml-pipeline.html). However, we will not be covering those in this notebook. If you are interested in combining Spark and scikit-learn you may want to read [this](https://databricks.com/blog/2016/02/08/auto-scaling-scikit-learn-with-apache-spark.html).

### Predicting delays 

In the next step we will add an extra column to the dataframe which indicates whether a flight had a delay of more than 15 minutes or not. When so, we enter a value of 1.0 in the column 'label', otherwise we use '0.0'.

We will try to predict the value of this 'label' column by using Random Forests later. 

In the next cell we use [withColumn](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.withColumn) to add an extra column to a data frame. Note that we do not 'change' or 'edit' an existing dataframe but that we create a new one. Remember that data frames are immutable.

The [when](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.when) condition, together with 'otherwise', functions like an if-then-else statement.


In [ ]:
from pyspark.sql import functions as F

OrdDelayeddf = orddf.withColumn('label', F.when(orddf['DepDelay'] > 15, 1.0).otherwise(0.0))
OrdDelayeddf.show()

### Dealing with categorical variables - using StringIndexer

The feature (or column) 'UniqueCarrier' is a categorical feature. Our Random Forest Classifier requires that we map the values of a categorical feature to numbers. Spark ML offers a function called [StringIndexer](http://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.StringIndexer) that does exactly this.

Let's see how it works in a toy example first.

In the next cell we create a dataframe called mldf. It has two columns, that we name 'id' and 'category'. Next we transform the mldf dataframe into a new data frame called 'index' by using a StringIndexer. The StringIndexer is given the input and an output column and then transforms the 'mldf' data frame. Run the cell and see if you understand what happens.

In [ ]:
from pyspark.ml.feature import StringIndexer

mldf = spark.createDataFrame(
    [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],
    ["id", "category"])
indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
indexed = indexer.fit(mldf).transform(mldf)
indexed.show()

### Using StringIndexer to transform 'UniqueCarrier'

Next, use StringIndexer yourself to transform 'UniqueCarrier'. As name of the output column use 'Carrier'! We will be using this name later on. 

In [ ]:
#TODO
# Replace <FILL IN> with code of your own 
stringIndexer = StringIndexer(<FILL IN>)

si_model = stringIndexer.fit(OrdDelayeddf)
si_df = si_model.transform(OrdDelayeddf)
si_df.show()



### Transforming the 'Dest' column.

We will do the same for the 'Dest' column.

In [ ]:
stringIndexer = StringIndexer(inputCol='Dest', outputCol='Destination')
si_model = stringIndexer.fit(si_df)
dest_model_df = si_model.transform(si_df)

### Creating a feature vector: using VectorAssembler

All features that are used for building the classification model should be assembled in a feature vector. And again, Spark has a method for doing this: [VectorAssembler](http://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.VectorAssembler)

And again, let's see a toy example first.

First, we create a data frame with two rows and three columns, named 'a', 'b' and 'c'. Suppose now, that from this data frame we only want to use column 'a' and 'c' as features. Then we use only these columns as values for 'inputCols' in VectorAssembler. Note that the result of the transformation is a new data frame with an extra column called 'features'. This contains the features of the columns we selected, in the proper format.

In [ ]:
from pyspark.ml.feature import VectorAssembler

mldf = spark.createDataFrame([(1, 0, 3), (2,3,2)], ["a", "b", "c"])
vecAssembler = VectorAssembler(inputCols=['a','c'], outputCol="features")
ass = vecAssembler.transform(mldf)
ass.show()

### Using a VectorAssembler
 
 In the cell below build the VectorAssembler that can be used to transform 'dest_model_df' in a data frame that cointains a column called 'features' (we reuse this name!). As input use the featureCols list.

In [ ]:
#TODO
# Replace <FILL IN> with code of your own 

featureCols = ['Month', 'DayofMonth', 'DayofWeek', 'Distance', 'Carrier', 'Destination']

#set the input and output column names
assembler = VectorAssembler<FILL IN>

# return a dataframe with all of the  feature columns in  a vector column
features_df = assembler.transform(dest_model_df)
features_df.show()

### Splitting the data set into a part for training and testing

Next, we split the data set into two parts. 80 percent of the data will be in the training set and the remaining 20 percent will be our test set. The assignment of records to these two sets is random. We use the ['RandomSplit'](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) method that needs a list of weights and a seed for the random generation of the splits. 

In [ ]:
train_data, test_data = features_df.randomSplit([0.8, 0.2], 12345)
train_data.show()

### Training the Random Forest Classifier

Now we are ready to train the Random Forest Classifier. As input we have to provide a few things: the number of trees we want to create, the depth of each tree, the label column (the class we want to predict). In addition, we provide a seed for random generation and a maxBins number for categorical variables.

Spark ML offers ways to search for the optimal set of configuration parameters. See here: http://spark.apache.org/docs/latest/ml-tuning.html . Running these, can be quite slow in local mode.

So let's run the classifier with a limited number of trees, and with shallow depth. If we want to see the tree we can use the toDebugString method. The training may take a while.


In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(numTrees=5, maxBins= 150, maxDepth=4, labelCol="label", seed=42)
model = rf.fit(train_data)
#model.toDebugString


### Predictions on the test data

Let's use the model to make predictions on the test_data. Note, that by predicting we transform one data frame into another.

In [ ]:
predictions = model.transform(test_data)
predictions.show()

### Evaluating the results

So is machine learning that easy? Yes... but wait till we see how good the model performs.

We use Sparks [BinaryClassificationEvaluator](http://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.BinaryClassificationEvaluator) to get some information on how the model performs. In this case, we measure accuracy. Check, the documentation for more evaluation metrics.




In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol='label')
accuracy = evaluator.evaluate(predictions) 
accuracy

### And now...

This is not really great but it's better than chance. We can tweak the model by training again, or we may choose other features, or add some more information to our features. For example, what about holidays? Or, if we can get weather data that could improve our model significantly. Maybe you want to look at other models for classification? The journey does not end here, but really has just begun.

But for now, we leave it at this. 